The lb_best is from [@V.B.](https://www.kaggle.com/code/vyacheslavbolotin/ps-s5e3-rainfall-division-attention).The condition is inspired from [@Iritaco2533](https://www.kaggle.com/competitions/playground-series-s5e3/discussion/566054#3145146)

[**Discussion:** AUC post-processing can improve your CV](https://www.kaggle.com/competitions/playground-series-s5e3/discussion/567310)

k=1.001 LB=0.90158

k=1.002 LB=0.90131

K=1.005 LB=0.90185

k=1.01  LB=0.90077




**This notebook is just for demonstration and discussion with @Data User, high LB is completely meaningless.**


In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
lb_best = pd.read_csv("/kaggle/input/ps-s5e3-rainfall-division-attention/submission.csv")
#0.90158
ori = pd.read_csv("/kaggle/input/hongkongrainfall/hongkong.csv",encoding="gbk")
test = pd.read_csv("/kaggle/input/playground-series-s5e3/test.csv").drop("id",axis=1)
test['winddirection'].fillna(test['winddirection'].median(), inplace=True)

<ipython-input-2-5e0763ddf16f>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test['winddirection'].fillna(test['winddirection'].median(), inplace=True)


In [3]:
tmp = lb_best['rainfall'].copy()
tmp.head(5)

0    0.973541
1    0.978166
2    0.812125
3    0.160289
4    0.091565
Name: rainfall, dtype: float64

In [4]:
condition1 =  (test.cloud > 73.5) & (test.sunshine < 0.5) & (test.pressure<=1020.35)&(test.windspeed>20.35) 
print(len(lb_best['rainfall'][condition1]))
lb_best['rainfall'][condition1] *= 1.005

# condition2 =  (test.humidity<81.924)&(test.cloud<=71.02)&(test.sunshine>9.661)
# lb_best['rainfall'][condition2] /= 1.005

103


<ipython-input-4-4a7de0f450e0>:3: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  lb_best['rainfall'][condition1] *= 1.005
<ipython-input-4-4a7de0f450e0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

In [5]:
lb_best['rainfall'].head(3)

0    0.978408
1    0.983057
2    0.812125
Name: rainfall, dtype: float64

In [6]:
lb_best['rainfall'][25],lb_best['rainfall'][29],lb_best['rainfall'][120],lb_best['rainfall'][123],lb_best['rainfall'][125]

(0.3547489782441453,
 0.4657780257671781,
 0.6033959972479139,
 0.4406655473105725,
 0.612546346182323)

In [7]:
condition1[25],condition1[29],condition1[120],condition1[123],condition1[125]

(False, False, False, False, False)

In [8]:
lb_best['rainfall'][15]=-1
lb_best['rainfall'][25]=-1 #id 2215 should be 25, not 15 
#rank 23
lb_best['rainfall'][29]=2 # rank 25
lb_best['rainfall'][120]=-1 #rank 26
lb_best['rainfall'][123]=2 #rank 24
lb_best['rainfall'][125]=2 #rank 27

<ipython-input-8-ae125e115579>:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  lb_best['rainfall'][15]=-1
<ipython-input-8-ae125e115579>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the 

In [9]:
from scipy.stats import rankdata
print((rankdata(tmp[:146])!=rankdata(lb_best['rainfall'][:146])).sum())
lb_best.to_csv('submission.csv', index=False)
lb_best.head()

144


,id,rainfall
0,2190,0.978408
1,2191,0.983057
2,2192,0.812125
3,2193,0.160289
4,2194,0.091565


## See how it has changed


In [10]:
def multi_k(df,k):
    df = df.copy()
    df[condition1] *= k
    return df

In [11]:
import numpy as np
import matplotlib.pyplot as plt

A = rankdata(tmp[:146])

B1 = rankdata(multi_k(lb_best['rainfall'],1.01)[:146])
B=B1
B2 = rankdata(multi_k(lb_best['rainfall'],1.005)[:146])
B3 = rankdata(multi_k(lb_best['rainfall'],1.002)[:146])
B4 = rankdata(multi_k(lb_best['rainfall'],1.001)[:146])

sorted_indices = np.argsort(A)
A = A[sorted_indices]
B1 = B1[sorted_indices]
B2 = B2[sorted_indices]
B3 = B3[sorted_indices]
B4 = B4[sorted_indices]
#print(np.array(tmp[:146][sorted_indices])[24])
plt.figure(figsize=(60, 100))
plt.subplot(5,1, 1)
plt.title("Array A")
bars_A = plt.bar(range(len(A)), A, color='skyblue')
plt.xticks(range(len(A)), [f'Pos {i}' for i in range(len(A))])
plt.ylabel('Value')
for bar in bars_A:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, height + 0.1, f'{height:.0f}', ha='center')
plt.axvline(24.5,color='red')
plt.subplot(5, 1, 2)
plt.title("K=1.01 LB 90077", fontsize=80)
bars_B = plt.bar(range(len(B)), B1, color='lightgreen')
plt.xticks(range(len(B)), [f'Pos {i}' for i in range(len(B))])
plt.ylabel('Value')

for bar in bars_B:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, height + 0.1, f'{height:.0f}', ha='center')
plt.axvline(24.5,color='red')

plt.subplot(5, 1, 3)
plt.title("K=1.005 LB 90185", fontsize=80)
bars_B = plt.bar(range(len(B)), B2, color='lightgreen')
plt.xticks(range(len(B)), [f'Pos {i}' for i in range(len(B))])
plt.ylabel('Value')

for bar in bars_B:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, height + 0.1, f'{height:.0f}', ha='center')
plt.axvline(24.5,color='red')
plt.subplot(5, 1, 4)
plt.title("K=1.002 LB 90131", fontsize=80)
bars_B = plt.bar(range(len(B)), B3, color='lightgreen')
plt.xticks(range(len(B)), [f'Pos {i}' for i in range(len(B))])
plt.ylabel('Value')

for bar in bars_B:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, height + 0.1, f'{height:.0f}', ha='center')
plt.axvline(24.5,color='red')
plt.subplot(5, 1, 5)
plt.title("K=1.001 LB 90158", fontsize=80)
bars_B = plt.bar(range(len(B)), B4, color='lightgreen')
plt.xticks(range(len(B)), [f'Pos {i}' for i in range(len(B))])
plt.ylabel('Value')

for bar in bars_B:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, height + 0.1, f'{height:.0f}', ha='center')
plt.axvline(24.5,color='red')

# for i in range(len(A)):
#     if A[i] != B[i]:
#         plt.annotate(f'{A[i]}→{B[i]}', xy=(i, A[i]), xytext=(i, B[i]),
#                      arrowprops=dict(facecolor='black', shrink=0.05))

plt.tight_layout()
plt.savefig("1.jpg")
plt.show()
